# Model building Pipeline using easyflow feature_encoders module

This module is a fusion between keras layers and tensorflow feature columns

In [1]:
import pandas as pd
import tensorflow as tf

In [2]:
# local imports
from easyflow.data.mapper import TensorflowDataMapper
from easyflow.feature_encoders.transformer import FeatureColumnTransformer, FeatureUnionTransformer
from easyflow.feature_encoders.feature_encoder import NumericalFeatureEncoder, EmbeddingFeatureEncoder, CategoricalFeatureEncoder

In [3]:
CSV_HEADER = [
    "age",
    "workclass",
    "fnlwgt",
    "education",
    "education_num",
    "marital_status",
    "occupation",
    "relationship",
    "race",
    "gender",
    "capital_gain",
    "capital_loss",
    "hours_per_week",
    "native_country",
    "income_bracket",
]

data_url = (
    "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
)

try:
    data_frame = pd.read_csv('adult_features.csv')
    labels_binary = pd.read_csv('adult_labels.csv')
except:
    data_frame = pd.read_csv(data_url, header=None, names=CSV_HEADER)
    labels = data_frame.pop("income_bracket")
    labels_binary = 1.0 * (labels == " >50K")
    data_frame.to_csv('adult_features.csv', index=False)
    labels_binary.to_csv('adult_labels.csv', index=False)

print(f"Train dataset shape: {data_frame.shape}")

Train dataset shape: (32561, 14)


In [4]:
batch_size = 256
dataset_mapper = TensorflowDataMapper() 
dataset = dataset_mapper.map(data_frame, labels_binary)

train_data_set, val_data_set = dataset_mapper.split_data_set(dataset)
train_data_set = train_data_set.batch(batch_size)
val_data_set = val_data_set.batch(batch_size)

## Set up the feature encoding list

In [5]:
NUMERIC_FEATURE_NAMES = [
    "age",
    "education_num",
    "capital_gain",
    "capital_loss",
    "hours_per_week",
]

CATEGORICAL_FEATURES_NAMES = [
    "workclass",
    "marital_status",
    "relationship",
    "race",
    "gender"]

EMBEDDING_FEATURES_NAMES = ['education',
                            'occupation',
                            'native_country']

In [6]:
feature_encoder_list = [('numerical_features', NumericalFeatureEncoder(), NUMERIC_FEATURE_NAMES),
                        ('categorical_features', CategoricalFeatureEncoder(), CATEGORICAL_FEATURES_NAMES),
                        ('embedding_features_deep', EmbeddingFeatureEncoder(), EMBEDDING_FEATURES_NAMES),
                        ('embedding_features_wide', CategoricalFeatureEncoder(), EMBEDDING_FEATURES_NAMES)]

## Setting up feature layer and feature encoders

There are two main column transformer classes namely FeatureColumnTransformer and FeatureUnionTransformer. For this example we are going to build a Wide and Deep model architecture. So we will be using the FeatureColumnTransformer since it gives us more flexibility. FeatureUnionTransformer concatenates all the features in the input layer

In [7]:
feature_layer_inputs, feature_encoders =  FeatureColumnTransformer(feature_encoder_list).transform(train_data_set)

In [8]:
deep_features = feature_encoders['numerical_features']+\
                feature_encoders['categorical_features']+\
                feature_encoders['embedding_features_deep']

wide_features = feature_encoders['embedding_features_wide']


In [9]:
deep = tf.keras.layers.DenseFeatures(deep_features)(feature_layer_inputs)
deep = tf.keras.layers.BatchNormalization()(deep)

wide = tf.keras.layers.DenseFeatures(wide_features)(feature_layer_inputs)

for nodes in [128, 64, 32]:
    deep = tf.keras.layers.Dense(nodes, activation='relu')(deep)
    deep = tf.keras.layers.Dropout(0.5)(deep)

wide_and_deep = tf.keras.layers.concatenate([deep, wide])
output = tf.keras.layers.Dense(1, activation='sigmoid')(wide_and_deep)
model = tf.keras.Model(inputs=[v for v in feature_layer_inputs.values()], outputs=output)
model.compile(loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.0),
              optimizer=tf.keras.optimizers.Adam(lr=0.001),
              metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy'), tf.keras.metrics.AUC(name='auc')])

In [10]:
model.fit(train_data_set, validation_data=val_data_set, epochs=10)

Epoch 1/10


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/functional.py:543: UserWarning: Input dict contained keys ['fnlwgt'] which did not match any model input. They will be ignored by the model.
  [n for n in tensors.keys() if n not in ref_input_names])


96/96 [==============================] - 2s 17ms/step - loss: 0.4817 - accuracy: 0.7550 - auc: 0.7550 - val_loss: 0.5554 - val_accuracy: 0.6660 - val_auc: 0.8772
Epoch 2/10
96/96 [==============================] - 1s 10ms/step - loss: 0.3782 - accuracy: 0.8252 - auc: 0.8694 - val_loss: 0.4102 - val_accuracy: 0.7885 - val_auc: 0.8934
Epoch 3/10
96/96 [==============================] - 1s 10ms/step - loss: 0.3645 - accuracy: 0.8320 - auc: 0.8801 - val_loss: 0.3445 - val_accuracy: 0.8382 - val_auc: 0.9038
Epoch 4/10
96/96 [==============================] - 1s 9ms/step - loss: 0.3535 - accuracy: 0.8381 - auc: 0.8868 - val_loss: 0.3282 - val_accuracy: 0.8482 - val_auc: 0.9057
Epoch 5/10
96/96 [==============================] - 1s 10ms/step - loss: 0.3463 - accuracy: 0.8431 - auc: 0.8921 - val_loss: 0.3240 - val_accuracy: 0.8521 - val_auc: 0.9086
Epoch 6/10
96/96 [==============================] - 1s 11ms/step - loss: 0.3478 - accuracy: 0.8433 - auc: 0.8900 - val_loss: 0.3094 - val_accuracy: